<a href="https://colab.research.google.com/github/fabiomatricardi/buildLLMapplicationFree/blob/main/Learn_LangChain_GradioLLM_FreeAI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tests to create FREE AI AGENTS

In [ ]:
%pip install --upgrade --quiet  gradio_tools huggingface_hub langchain langchain-community langgraph google-search-results faiss-cpu tiktoken duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s

In [ ]:
from tqdm.rich import trange, tqdm
from rich.markdown import Markdown
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=90)
from typing import Any, Dict, Iterator, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from gradio_client import Client
from gradio_client import Client
from langchain_core.prompts import PromptTemplate

## ReACT examples with Chains - Using Qwen72B

For example, let’s say we want to ask our model for some up-to-date information about the upcoming
Olympic games. To do so, we are going to build a smart LangChain agent (as described in Chapter 2)
leveraging SerpAPIWrapperWrapper (to wrap the SerpApi to navigate the web), the AgentType tool
(to decide which type of agent to use for our goal), and other prompt-related modules (to make it
easier to “templatize” our instructions). Let’s see how we can do this (I won’t dive deeper into each
component of the following code since the next chapter will be entirely focused on LangChain and
its main components):

In [ ]:
from langchain_core.language_models.llms import LLM
class GradioLLMChat(LLM):
    """
    Custom LLM class based on the Gradio API call.
    """
    from typing import Any, Dict, Iterator, List, Mapping, Optional
    from langchain.callbacks.manager import CallbackManagerForLLMRun
    from langchain_core.language_models.llms import LLM
    from langchain_core.outputs import GenerationChunk
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import ChatPromptTemplate
    from gradio_client import Client
    chatbot: Any = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.chatbot = Client("https://qwen-qwen1-5-72b-chat.hf.space/--replicas/061qr/")

    @property
    def _llm_type(self) -> str:
        return "Gradio API client Qwen1.5-72b_Chat"

    def _call(
            self,
            prompt: str,
            stop: Optional[List[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            chatbot=None,
            request: float = 0.95, #it's the history touple
            param: str = 'You are a helpful assistant', #it's the system message
    ) -> str:
        """
        Make an API call to the Gradio API client Meta_llama3_8b using the specified prompt and return the response.
        """
        if chatbot is None:
            chatbot = self.chatbot

        # Return the response from the API
        result = chatbot.predict(   #.submit for streaming effect / .predict for normal output
                param,	# str  in 'Input' Textbox component
                [],	# Tuple[str | Dict(file: filepath, alt_text: str | None) | None, str | Dict(file: filepath, alt_text: str | None) | None]  in 'Qwen1.5-72B-Chat' Chatbot component
                prompt,	# str  in 'parameter_9' Textbox component
                api_name="/model_chat"
        )
        return result[1][0][1]

## GradioLLM + DuckDuckGo AI agent tool

In [ ]:
import os
#from https://python.langchain.com/v0.1/docs/integrations/tools/ddg/
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
#wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=8)
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.schema import HumanMessage
from gradio_client import Client

model = GradioLLMChat()
search = DuckDuckGoSearchAPIWrapper(max_results=5)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events, search online news"
    )
    ]

agent_executor = initialize_agent(tools, model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

Loaded as API: https://qwen-qwen1-5-72b-chat.hf.space/--replicas/061qr/ ✔


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
console.print(agent_executor.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search(query: str) -> str - useful for when you need to answer questions about current 
events, search online news

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

In [ ]:
agent_executor('search online the news and tell me the incident on May 2024 when Giorgia Meloni met governor De Luca')



> Entering new AgentExecutor chain...
I need to use the search function to find relevant information about this specific event.
Action: Search
Action Input: "May 2024 Giorgia Meloni governor De Luca incident"
Observation: Leggi su Sky TG24 l'articolo De Luca risponde a Meloni dopo incontro Caivano: "Ha comunicato sua vera identità" ... 29 mag 2024 - 14:21 ... Dal presidente del Consiglio Giorgia Meloni, ha ... "Presidente De Luca, la stronza della Meloni!''. La premier stringe la mano al governatore della Campania così, ricordandogli come il … Nel corso di Tagadà, su La7, viene mandato il passaggio delle comunicazioni social di De Luca che a mente fredda contrattacca ad alzo zero contro Meloni. Ecco i rosicon...zi contro Giorgia. Da De ... Caivano (Napoli), 28 maggio 2024 - È ancora lite tra la premier Giorgia Meloni e Vincenzo De Luca, presidente della Regione Campania. L'occasione, questa volta, la offre l'inaugurazione del ... Premier Giorgia Meloni introduced herself as "that bit

{'input': 'search online the news and tell me the incident on May 2024 when Giorgia Meloni met governor De Luca',
 'output': 'In May 2024, an incident occurred between Italian Prime Minister Giorgia Meloni and Campania Governor Vincenzo De Luca during an inauguration event in Caivano, Naples. Governor De Luca had previously referred to Meloni as a "stronza" (bitch) in a public statement, which led to a tense interaction between the two when they met. During their encounter, Meloni reportedly introduced herself to De Luca as "that bitch," referencing his previous comment. This exchange sparked further controversy and media attention.'}